In [ ]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / 'src'

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.extraction import (
    get_images_lists_from_path,
    get_images_lists_from_more_paths,
    delete_bad_dirs_brats2020
)

from src.plots import (
    plot_observation,
    plot_deeplab_mobile_predictions
)

from src.deeplab_mobile.modelling import(
    select_images_input,
    get_deeplab_mobile_model,
    train_deeplab_mobile
)

from src.deeplab_mobile.segdataset import(
    get_mobile_dataloaders
)

from src.utils import(
    LOGS_FILE_PATH
)

%load_ext autoreload
%autoreload 2

In [ ]:
hgg = Path(os.getcwd()) / 'data' / 'HGG'
lgg = Path(os.getcwd()) / 'data' / 'LGG'
two = Path(os.getcwd()) / 'data' / 'brats2020'
hgg_2018 = Path(os.getcwd()) / 'data' / 'brats2018' / 'HGG'
lgg_2018 = Path(os.getcwd()) / 'data' / 'brats2018' / 'LGG'

In [ ]:
bad_dirs = delete_bad_dirs_brats2020(two)

In [ ]:
t2, t1ce, t1, flair, seg = get_images_lists_from_path(hgg)
t2l, t1cel, t1l, flairl, segl = get_images_lists_from_path(lgg)

type_names = ['t2', 't1', 't1ce', 'flair', 'seg']
images = [t2, t1, t1ce, flair, seg]
imagesl = [t2l, t1l, t1cel, flairl, segl]
all_images = get_images_lists_from_more_paths([hgg, lgg, two, hgg_2018, lgg_2018])

In [ ]:
print(len(all_images[0]), 'images used')

In [ ]:
plot_observation(all_images, 0)

In [ ]:
plot_observation(all_images, -1)

In [ ]:
plot_observation(all_images, 600)

In [ ]:
type_to_use = 'flair'
images_chosen, images_seg = select_images_input(all_images, type_names, type_to_use = type_to_use)

In [ ]:
model = get_deeplab_mobile_model() 

In [ ]:
another = models.segmentation.deeplabv3_mobilenet_v3_large(
    pretrained=False,
    progress=True,
    num_classes = 3
)

In [ ]:
dataloaders = get_mobile_dataloaders(images_chosen, images_seg)

In [ ]:
train_deeplab_mobile(model, dataloaders, num_epochs=42)

In [ ]:
model_exp_name = 'flair_totalpipe_nighttrain'
torch.save(model, str(Path(os.getcwd()) / 'models' / (model_exp_name + '.pt')))

In [ ]:
modelname = 'flair_totalpipe_nighttrain.pt'
model = torch.load(Path(os.getcwd()) / 'models' / modelname)
model.eval()

In [ ]:
plot_deeplab_mobile_predictions(model, images)

In [ ]:
df = pd.read_csv(LOGS_FILE_PATH)

In [ ]:
df.plot(x='epoch',figsize=(15,8))